In [31]:
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv('creditcard.csv')

In [33]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [35]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [36]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [37]:
legit = df[df.Class==0]
fraud = df[df.Class==1]

In [38]:
legit.shape

(284315, 31)

In [39]:
fraud.shape

(492, 31)

In [40]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [41]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [42]:
df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [43]:
new_legit = legit.sample(n=492)

In [44]:
new_legit

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
167434,118682.0,-0.172717,0.980743,-1.319590,-1.171225,2.199315,-0.328742,2.028237,-0.354474,-0.986886,...,0.419096,1.243899,-0.597456,-0.270635,0.781906,-0.253125,0.095786,0.136903,48.42,0
243258,151857.0,1.930399,-0.357777,-1.438415,0.004841,0.701152,0.891335,-0.242953,0.339443,0.492133,...,-0.210101,-0.461457,0.345205,-1.663085,-0.498880,0.313636,-0.033709,-0.089284,4.49,0
105700,69653.0,-0.927131,1.451459,0.792693,0.065033,-0.426104,-1.076085,0.281494,0.420009,-0.215180,...,-0.265450,-0.810274,0.052129,0.297292,-0.121859,0.082640,0.228320,0.083092,8.99,0
10139,15478.0,-1.055318,-0.406117,2.181585,-1.337705,0.216771,-1.464110,0.266623,-0.358862,2.437841,...,-0.122870,0.059087,-0.199684,0.716826,0.557306,0.086593,-0.178856,-0.152722,37.46,0
63080,50544.0,-0.601334,1.172203,1.636770,1.123101,0.361950,0.294057,0.753862,-0.026369,-0.387869,...,-0.045164,0.510836,-0.199098,0.275528,-0.089837,-0.280663,0.380566,0.036650,4.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12379,21693.0,1.228059,-0.282108,1.296592,0.272377,-1.199034,-0.317642,-1.015511,0.050034,2.301372,...,0.018483,0.277405,-0.034361,0.018924,0.055892,1.098973,-0.063459,0.012085,17.95,0
130713,79414.0,1.098126,-0.237990,0.021788,-0.045117,-0.399525,-0.733752,0.151845,-0.068971,-0.097181,...,-0.450589,-1.664452,0.169859,-0.019393,-0.099530,0.508799,-0.120975,0.008730,95.94,0
195751,131206.0,-1.717685,1.393168,-1.360300,-0.043114,0.028088,-1.212531,1.045176,0.122259,0.147657,...,-0.118767,-0.130739,0.134577,-0.126567,-0.658522,-0.569720,-0.457446,-0.195978,68.73,0
1156,900.0,-1.239276,0.941957,1.008600,-0.624973,-0.150140,-0.296631,0.063537,0.507260,0.038386,...,0.065438,0.258003,-0.283958,0.090616,-0.106607,0.478273,-0.035538,0.056306,1.00,0


In [45]:
new_df = pd.concat([new_legit,fraud], axis=0)

In [46]:
new_df.sample(6)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
225138,144122.0,2.052091,-0.157074,-1.094440,0.422208,-0.232626,-1.161020,0.079112,-0.232797,0.743621,...,-0.298736,-0.785709,0.362920,-0.050034,-0.359022,0.205549,-0.076387,-0.063353,1.98,0
262068,160294.0,-0.576504,0.333119,-0.495374,-3.478498,-0.172836,-1.119555,0.317621,0.049337,-0.155920,...,0.028632,0.796879,-0.383372,-0.576659,0.100909,0.004375,0.192187,0.154613,15.17,0
192584,129764.0,-2.434004,3.225947,-6.596282,3.593161,-1.079452,-1.739741,-0.047420,0.301424,-1.779434,...,-0.035491,-0.419178,0.157436,-0.714849,0.468859,-0.348522,0.420036,-0.327643,362.55,1
151519,95628.0,-17.518909,12.572118,-19.038538,11.190895,-13.554721,-0.411924,-23.189397,-5.301412,-8.630390,...,-4.969478,0.976125,1.841248,0.334418,-0.720128,-0.232603,-3.021992,-0.478158,1.63,1
64329,51112.0,-9.848776,7.365546,-12.898538,4.273323,-7.611991,-3.427045,-8.350808,6.863604,-2.387567,...,0.931958,-0.874467,-0.192639,-0.035426,0.538665,-0.263934,1.134095,0.225973,99.99,1
30442,35926.0,-3.896583,4.518355,-4.454027,5.547453,-4.121459,-1.163407,-6.805053,2.928356,-4.917130,...,1.691042,0.920021,-0.151104,0.011007,0.080303,0.412191,0.635789,0.501050,4.56,1


In [47]:
new_df['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [48]:
new_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94804.018293,-0.056249,-0.023090,-0.041168,-0.045505,0.085312,-0.004051,0.004972,0.042090,-0.009117,...,0.024207,0.004361,-0.038757,0.049458,-0.019533,-0.026359,0.011475,0.028388,-0.006131,88.451382
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321
